# Introduction
- [NLP领域中的token和tokenization](https://www.zhihu.com/question/64984731/answer/3183726323)
- [从词到数：Tokenizer与Embedding串讲](https://zhuanlan.zhihu.com/p/631463712)
- Tokenization（分词） 在自然语言处理(NLP)的任务中是最基本的一步，把文本内容处理为最小基本单元即token用于后续的处理
- 把文本处理成token有一系列的方法，基本思想是构建一个词表通过词表一一映射进行分词，以下以分词粒度为角度进行介绍如何构建合适的词表

# word（词）粒度
- 英语通过天然的分隔符进行分词，中文通过如jieba等进行分词
- 优点：语义明确；上下文理解
- 缺点：长尾效应和稀有词问题；OOV的问题；形态关系和词缀关系

# char（字符）粒度
- 字符为最小基本单元进行分词
- 优点：统一处理方式；解决OOV问题
- 缺点：语义信息不明确；处理效率低

# subword(子词)粒度
- 既不希望将文本切分成单独的词（太大），也不想将其切分成单个字符（太小），而是希望得到介于词和字符之间的子词单元
- 在BERT时代，`WordPiece` 分词方法被广泛应用，比如 BERT、DistilBERT等
- 在大语言模型时代，最常用的分词方法是`Byte-Pair Encoding (BPE)`和`Byte-level BPE(BBPE)`
- `Byte-Pair Encoding (BPE)`最初是一种文本压缩算法在15年被引入到NLP用于分词，GPT，RoBERTa等都采用了这种分词方法
- `Byte-level BPE(BBPE)`是19年在BPE的基础上提出以Byte-level(字节)为粒度的分词方法，目前 GPT2，BLOOM，Llama，Falcon等采用该分词方法

## WordPiece
- 核心思想是将单词拆分成多个前缀符号（比如BERT中的##）最小单元，再通过子词合并规则将最小单元进行合并为子词级别
- 核心步骤如下：
  - 计算初始词表
  - 计算合并分数：也称作互信息（信息论中衡量两个变量之间的关联程度），公式为`分数 = 合并pair候选的频率 / (第一个元素的频率 × 第二个元素的频率)`
  - 合并分数最高的子词对
  - 重复合并步骤
  - 分词

In [1]:
sentences = [
    "我",
    "喜欢",
    "吃",
    "苹果",
    "他",
    "不",
    "喜欢",
    "吃",
    "苹果派",
    "I like to eat apples",
    "She has a cute cat",
    "you are very cute",
    "give you a hug",
]

In [17]:
# 统计每个词出现的频率并初始化初始词表
from collections import defaultdict

def bulid_stats(sentences):
    stats = defaultdict(int) #所有键的值默认为0
    for sentence in sentences:
        symbols = sentence.split() #使用空格作为分隔符  
        for symbol in symbols:
            stats[symbol]+=1
    return stats

stats = bulid_stats(sentences)
print("stats:",stats)

alphabet = []
for word in stats.keys():
    if word[0] not in alphabet:
        alphabet.append(word[0])
    for letter in word[1:]:
        if f"##{letter}" not in alphabet: # 
            alphabet.append(f"##{letter}")
alphabet.sort()

vocab = alphabet.copy()
print("alphabet:",alphabet)

# 根据初始词表拆分每个词
splits = {
    word:[c if i==0 else f"##{c}" for i,c in  enumerate(word)]
    for word in stats.keys()
}    
print("splits:", splits)

stats: defaultdict(<class 'int'>, {'我': 1, '喜欢': 2, '吃': 2, '苹果': 1, '他': 1, '不': 1, '苹果派': 1, 'I': 1, 'like': 1, 'to': 1, 'eat': 1, 'apples': 1, 'She': 1, 'has': 1, 'a': 2, 'cute': 2, 'cat': 1, 'you': 2, 'are': 1, 'very': 1, 'give': 1, 'hug': 1})
alphabet: ['##a', '##e', '##g', '##h', '##i', '##k', '##l', '##o', '##p', '##r', '##s', '##t', '##u', '##v', '##y', '##果', '##欢', '##派', 'I', 'S', 'a', 'c', 'e', 'g', 'h', 'l', 't', 'v', 'y', '不', '他', '吃', '喜', '我', '苹']


In [29]:
# 根据上述提到的计算互信息的分数公式进行计算
def compute_pair_scores(splits):
    letter_freqs = defaultdict(int)
    pair_freqs = defaultdict(int)
    for word,freq in stats.items():
        split = splits[word]
        if len(split) == 1:
            letter_freqs[split[0]]+=freq
            continue
        for i in range(len(split)-1):
            pair = (split[i],split[i+1])
            letter_freqs[split[i]]+=freq
            pair_freqs[pair]+=freq
        letter_freqs[split[-1]]+=freq

    score = {
        pair: freq/(letter_freqs[pair[0]]*letter_freqs[pair[1]])
        for pair, freq in pair_freqs.items()
    }
    return score



pair_scores = compute_pair_scores(splits)
for i, key in enumerate(pair_scores.keys()):
    print(f"{key}: {pair_scores[key]}")
    if i >= 5:
        break
    
# 将分数最高的进行合并然后开始循环迭代，看一看分数最高的pair（子词对)
best_pair = ""
max_score = 0
for pair, score in pair_scores.items():
    if score > max_score:
        best_pair = pair
        max_score = score
print(best_pair,max_score)

('喜', '##欢'): 0.5
('苹', '##果'): 0.5
('##果', '##派'): 0.5
('l', '##i'): 0.5
('##i', '##k'): 0.5
('##k', '##e'): 0.125


In [33]:
# 最后就是一直进行循环迭代，直到vocab达到了我们想要的数量
def merge_pair(pair1,pair2,splits):
    for word in stats:
        split = splits[word]
        if len(split) == 1:
            continue
        i = 0
        while i < len(split) - 1:
            if split[i] == pair1 and split[i+1] == pair2:
                merge = pair1 + pair2[2:] if pair2.startswith("##") else pair1+pair2
                split = split[:i] + [merge] + split[i+2:]
            else:
                i += 1
        splits[word] = split
    return splits  

vocab_size = 50
while len(vocab) < vocab_size:
    score = compute_pair_scores(splits)
    best_pair,max_score = "",0
    for pair,score in score.items():
        if  score > max_score:
            best_pair,max_score = pair,score
    splits = merge_pair(*best_pair,splits)
    new_token = (
        best_pair[0]+best_pair[1][2:]
        if best_pair[1].startswith('##') 
        else best_pair[0]+best_pair[1]
    )
    vocab.append(new_token)

print("vocab:", vocab)

all_vocab = vocab + ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"]# + other_alphabet

vocab: ['##a', '##e', '##g', '##h', '##i', '##k', '##l', '##o', '##p', '##r', '##s', '##t', '##u', '##v', '##y', '##果', '##欢', '##派', 'I', 'S', 'a', 'c', 'e', 'g', 'h', 'l', 't', 'v', 'y', '不', '他', '吃', '喜', '我', '苹', 'Sh', '喜欢', '苹果', '苹果派', 'li', 'lik', 'gi', 'giv', '##pl', '##ppl', '##ry', 'to', 'yo', 'ea', 'eat']


## Byte-Pair Encoding (BPE)
- 核心思想是逐步合并出现频率最高的子词对而不是像Wordpiece计算合并分数，从而构建出一个词汇表
- 核心步骤如下：
  - 计算初始词表
  - 构建频率统计
  - 合并频率最高的子词对
  - 重复合并步骤
  - 分词
- 当词汇表的大小受限时，一些较少频繁出现的子词和没有在训练过程中见过的子词，就会无法进入词汇表出现`OOV`

In [4]:
sentences = [
    "我",
    "喜欢",
    "吃",
    "苹果",
    "他",
    "不",
    "喜欢",
    "吃",
    "苹果派",
    "I like to eat apples",
    "She has a cute cat",
    "you are very cute",
    "give you a hug",
]

In [6]:
# 统计每个词出现的频率并初始化初始词表

# 构建频率统计
from  collections import defaultdict
def build_stats(sentences):
    stats = defaultdict(int)
    for sentence in sentences:
        symbols = sentence.split()
        for symbol in symbols:
            stats[symbol] += 1
    return stats
stats = build_stats(sentences)
print("stats:", stats)

alphabet = []
for word in stats.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()

# 初始词表
vocab = alphabet.copy()
print("alphabet:", alphabet)

# 根据初始词表拆分每个词，计算左右pair(子词对)出现的频率
splits = {word: [c for c in word]  for word in stats.keys()}
print('splits:',splits)

stats: defaultdict(<class 'int'>, {'我': 1, '喜欢': 2, '吃': 2, '苹果': 1, '他': 1, '不': 1, '苹果派': 1, 'I': 1, 'like': 1, 'to': 1, 'eat': 1, 'apples': 1, 'She': 1, 'has': 1, 'a': 2, 'cute': 2, 'cat': 1, 'you': 2, 'are': 1, 'very': 1, 'give': 1, 'hug': 1})
alphabet: ['I', 'S', 'a', 'c', 'e', 'g', 'h', 'i', 'k', 'l', 'o', 'p', 'r', 's', 't', 'u', 'v', 'y', '不', '他', '吃', '喜', '我', '果', '欢', '派', '苹']
splits: {'我': ['我'], '喜欢': ['喜', '欢'], '吃': ['吃'], '苹果': ['苹', '果'], '他': ['他'], '不': ['不'], '苹果派': ['苹', '果', '派'], 'I': ['I'], 'like': ['l', 'i', 'k', 'e'], 'to': ['t', 'o'], 'eat': ['e', 'a', 't'], 'apples': ['a', 'p', 'p', 'l', 'e', 's'], 'She': ['S', 'h', 'e'], 'has': ['h', 'a', 's'], 'a': ['a'], 'cute': ['c', 'u', 't', 'e'], 'cat': ['c', 'a', 't'], 'you': ['y', 'o', 'u'], 'are': ['a', 'r', 'e'], 'very': ['v', 'e', 'r', 'y'], 'give': ['g', 'i', 'v', 'e'], 'hug': ['h', 'u', 'g']}


In [9]:
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word,freq in stats.items():
        split = splits[word]
        if len(split)==1:
            continue
        for i in range(len(split)-1):
            pair_freqs[(split[i], split[i+1])] += freq
    return pair_freqs


pair_freqs = compute_pair_freqs(splits)
for i,key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

# 然后开始循环迭代找到出现频率最高的pair(子词对)
best_pair,max_freq="",0
for pair,freq in pair_freqs.items():
    if freq > max_freq:
        best_pair,max_freq = pair,freq
print(best_pair,max_freq)

('喜', '欢'): 2
('苹', '果'): 2
('果', '派'): 1
('l', 'i'): 1
('i', 'k'): 1
('k', 'e'): 1
('喜', '欢') 2


In [10]:
def merge_pair(a,b,splits):
    for word in stats.keys():
        split = splits[word]
        if len(split)==1:
            continue
        i = 0
        while i < len(split)-1:
            if split[i] == a and split[i+1] == b:
                split = split[:i]+[a+b]+split[i+2:]
            else: i+=1
        splits[word] = split
    return splits

# 一直进行循环直到vocab达到了我们想要的数量
merges = {}
vocab_size = 50

while len(vocab)<vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair,max_freq="",0
    for pair,freq in pair_freqs.items():
        if freq > max_freq:
            best_pair,max_freq = pair,freq
    splits = merge_pair(*best_pair,splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

print("merges:", merges)
print("vocab:", vocab)
print("splits:", splits)

all_vocab = vocab + ["[PAD]", "[UNK]", "[BOS]", "[EOS]"] #+ other_alphabet

merges: {('喜', '欢'): '喜欢', ('苹', '果'): '苹果', ('a', 't'): 'at', ('c', 'u'): 'cu', ('cu', 't'): 'cut', ('cut', 'e'): 'cute', ('y', 'o'): 'yo', ('yo', 'u'): 'you', ('v', 'e'): 've', ('苹果', '派'): '苹果派', ('l', 'i'): 'li', ('li', 'k'): 'lik', ('lik', 'e'): 'like', ('t', 'o'): 'to', ('e', 'at'): 'eat', ('a', 'p'): 'ap', ('ap', 'p'): 'app', ('app', 'l'): 'appl', ('appl', 'e'): 'apple', ('apple', 's'): 'apples', ('S', 'h'): 'Sh', ('Sh', 'e'): 'She', ('h', 'a'): 'ha'}
vocab: ['I', 'S', 'a', 'c', 'e', 'g', 'h', 'i', 'k', 'l', 'o', 'p', 'r', 's', 't', 'u', 'v', 'y', '不', '他', '吃', '喜', '我', '果', '欢', '派', '苹', '喜欢', '苹果', 'at', 'cu', 'cut', 'cute', 'yo', 'you', 've', '苹果派', 'li', 'lik', 'like', 'to', 'eat', 'ap', 'app', 'appl', 'apple', 'apples', 'Sh', 'She', 'ha']
splits: {'我': ['我'], '喜欢': ['喜欢'], '吃': ['吃'], '苹果': ['苹果'], '他': ['他'], '不': ['不'], '苹果派': ['苹果派'], 'I': ['I'], 'like': ['like'], 'to': ['to'], 'eat': ['eat'], 'apples': ['apples'], 'She': ['She'], 'has': ['ha', 's'], 'a': ['a'], 'cute

## Byte-level BPE(BBPE)
- Unicode 是字符集，为每个字符分配唯一的代码点
- UTF-8 是一种基于 Unicode 的字符编码方式，用于在计算机中存储和传输字符
- Byte(字节)：计算机存储和数据处理时，字节是最小的单位，一个字节包含8个(Bit)二进制位,一个字节能表示的范围是`256`
- `BPE`是最小词汇是字符级别，而`BBPE`是字节级别的，通过`UTF-8`的编码方式，理论上可以表示世界上所有字符
- 核心步骤和`BPE`除了实现粒度不一样，其他都一样:
  - 计算初始词表
  - 构建频率统计
  - 合并频率最高的子词对
  - 重复合并步骤
  - 分词
- 现在的大模型中，以`Byte-level BPE(BBPE)`这种方式进行分词是不会出现`OOV`
- 但词表中如果没有word级别的词的话，一些中英文就会分词分的很细碎(比如Llama),于是出现了扩充Llama中文词表的工作
  

In [1]:
from collections import defaultdict
sentences = [
    "我",
    "喜欢",
    "吃",
    "苹果",
    "他",
    "不",
    "喜欢",
    "吃",
    "苹果派",
    "I like to eat apples",
    "She has a cute cat",
    "you are very cute",
    "give you a hug",
]
# 构建初始词汇表，包含一个字节的256个表示
initial_vocab = [bytes([byte]) for byte in range(256)]
vocab = initial_vocab.copy()
print("initial_vocab:", initial_vocab)

# 构建频率统计
def build_stats(sentences):
    stats = defaultdict(int)
    for sentence in sentences:
        symbols = sentence.split()
        for symbol in symbols:
            stats[symbol.encode("utf-8")] += 1
    return stats
stats = build_stats(sentences)
print("stats:",stats)

splits = {word: [byte for byte in word] for word in stats.keys()}
print("splits:",splits)

def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word, freq in stats.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

# pair_freqs = compute_pair_freqs(splits)

def merge_pair(pair, splits):
    merged_byte = bytes(pair)
    for word in stats:
        split = splits[word]
        if len(split) == 1:
            continue
        i = 0
        while i < len(split) - 1:
            if split[i:i+2] == pair:  # 检查分割中是否有这对字节
                split = split[:i] + [merged_byte] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

vocab_size = 50
while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ()
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(best_pair, splits)
    merged_byte = bytes(best_pair)

print("vocab:", vocab)

initial_vocab: [b'\x00', b'\x01', b'\x02', b'\x03', b'\x04', b'\x05', b'\x06', b'\x07', b'\x08', b'\t', b'\n', b'\x0b', b'\x0c', b'\r', b'\x0e', b'\x0f', b'\x10', b'\x11', b'\x12', b'\x13', b'\x14', b'\x15', b'\x16', b'\x17', b'\x18', b'\x19', b'\x1a', b'\x1b', b'\x1c', b'\x1d', b'\x1e', b'\x1f', b' ', b'!', b'"', b'#', b'$', b'%', b'&', b"'", b'(', b')', b'*', b'+', b',', b'-', b'.', b'/', b'0', b'1', b'2', b'3', b'4', b'5', b'6', b'7', b'8', b'9', b':', b';', b'<', b'=', b'>', b'?', b'@', b'A', b'B', b'C', b'D', b'E', b'F', b'G', b'H', b'I', b'J', b'K', b'L', b'M', b'N', b'O', b'P', b'Q', b'R', b'S', b'T', b'U', b'V', b'W', b'X', b'Y', b'Z', b'[', b'\\', b']', b'^', b'_', b'`', b'a', b'b', b'c', b'd', b'e', b'f', b'g', b'h', b'i', b'j', b'k', b'l', b'm', b'n', b'o', b'p', b'q', b'r', b's', b't', b'u', b'v', b'w', b'x', b'y', b'z', b'{', b'|', b'}', b'~', b'\x7f', b'\x80', b'\x81', b'\x82', b'\x83', b'\x84', b'\x85', b'\x86', b'\x87', b'\x88', b'\x89', b'\x8a', b'\x8b', b'\x8c', b'\x8

# Embedding
## 中文的词向量
Chinese Word Vectors 中文词向量:https://github.com/Embedding/Chinese-Word-Vectors
## MTEB排行
MTEB: https://huggingface.co/spaces/mteb/leaderboard

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('aspire/acge_text_embedding')
texts1 = ["胡子长得太快怎么办？"]
texts2 = ["胡子长得快怎么办？", "怎样使胡子不浓密！", "香港买手表哪里好", "在杭州手机到哪里买"]
embs1 = model.encode(texts1, normalize_embeddings=True)
embs2 = model.encode(texts2, normalize_embeddings=True)
similarity = embs1 @ embs2.T
print(similarity) 

In [ ]:
# 获取 texts1[i] 对应的最相似 texts2[j]
for i in range(len(texts1)):
    scores = []
    for j in range(len(texts2)):
        scores.append([texts2[j], similarity[i][j]])
    scores = sorted(scores, key=lambda x:x[1], reverse=True)

    print(f"查询文本：{texts1[i]}")
    for text2, score in scores:
        print(f"相似文本：{text2}，打分：{score}")
    print() 

## milvus工具
- 向量数据库：专门为存储和检索向量数据而设计的数据库
- Milvus 是一款云原生向量数据库，它具备高可用、高性能、易拓展的特点，用于海量向量数据的实时召回
- [milvus官网](https://milvus.io/)
- [为AI而生的数据库：Milvus详解及实战](https://blog.csdn.net/lsb2002/article/details/132222947)

# Topic
- [NLP关键词提取必备：从TFIDF到KeyBert范式原理优缺点与开源实现](https://mp.weixin.qq.com/s/H2aRyF3tNZCCOxa4Oy2JMw)
- [盘点 KeyBert、TextRank 等九种主流关键词提取算法原理及 Python 代码实现](https://zhuanlan.zhihu.com/p/568271135)
- [NLP关键词提取方法总结及实现](https://blog.csdn.net/asialee_bird/article/details/96454544)
  
## 基于词袋加权的TFIDF算法

In [ ]:
import jieba.analyse
text = "美国总统拜登当地时间26日又语出惊人。据路透社报道，拜登当天在波兰的演 讲中，称俄罗斯总统普京“不能继续掌权了”。\
    白宫官员随后出来解释，克里姆林宫同日 也作出回应，称“这不是由拜登决定的”。报道称，在华沙皇家城堡发表的讲话中，拜登在\
    谴责普京后表示“看在上帝的份上，这个人不能继续掌权了”。对此，路透社表示，该言论引发了华盛顿方面对局势升级的担忧，美国\
    一直避免直接对乌克兰进行军事干预，并明确表示不支持政权更迭。随后，一名白宫官员对拜登这番话进行解释，称拜登的言论并不\
    代表华盛顿政策的转变，其目的是让世界为乌克兰问题上的长期冲突做好准备。该官员称，拜登意为“不能 允许普京对其邻国或该地区\
    行使权力”，而不是在讨论普京在俄罗斯的权力或俄 罗斯的政权 更迭问题。同日，路透社称，被问及拜登这一言论时，俄罗斯总统新闻\
    秘书佩斯科夫作出回应，称“这不是由拜登决定的。俄罗斯总统是由俄罗斯人选举产生的”。此前 ，俄罗斯安全会议副主席梅德韦杰夫接受\
    俄媒采访时曾表示，俄特别军事行动既定目标包括实现乌克兰去军事化、去纳粹化、成为中立国家、不奉行反俄政策。俄开展特别军事行动\
    首先是因为其设定的目标未能通过外交方式实现。他表示，社会调查显示，四分之三的俄罗斯民众支持在乌开展特别军事行动。西方国家企\
    图通过制裁影响俄民众，煽动他们反对政府，结果适得其反。"
# TF-IDF
jieba.analyse.extract_tags(text, topK=20, withWeight=True, allowPOS=('ns', 'n', 'vn', 'v','nr', 'nt'))

In [ ]:
# 输入语料库
corpus = ['this is the first document',
        'this is the second second document',
        'and the third one',
        'is this the first document']

words_list = list()
for i in range(len(corpus)):
    words_list.append(corpus[i].split(' '))
### 第一种 gensim
from gensim import corpora
# 赋给语料库中每个词(不重复的词)一个整数id
dic = corpora.Dictionary(words_list)
# print(dic.token2id)
# 元组中第一个元素是词语在词典中对应的id，第二个元素是词语在文档中出现的次数
new_corpus = [dic.doc2bow(words) for words in words_list]

# 训练模型并保存
from gensim import models
tfidf = models.TfidfModel(new_corpus)
tfidf_vec = []
for i in range(len(corpus)):
    string = corpus[i]
    string_bow = dic.doc2bow(string.lower().split())
    string_tfidf = tfidf[string_bow]
    tfidf_vec.append(string_tfidf)
# 输出 词语id与词语tfidf值
print(tfidf_vec)

# ### 第二种 sklearn
# from sklearn.feature_extraction.text import TfidfVectorizer
# tfidf_vec = TfidfVectorizer()
# tfidf_matrix = tfidf_vec.fit_transform(corpus)
# # 得到语料库所有不重复的词
# print(tfidf_vec.get_feature_names())
# # 得到每个单词对应的id值
# print(tfidf_vec.vocabulary_)
# # 得到每个句子所对应的向量，向量里数字的顺序是按照词语的id顺序来的
# print(tfidf_matrix.toarray())

## 考虑词关联网络的TextRank算法

In [ ]:
import jieba.analyse
text = "美国总统拜登当地时间26日又语出惊人。据路透社报道，拜登当天在波兰的演 讲中，称俄罗斯总统普京“不能继续掌权了”。\
    白宫官员随后出来解释，克里姆林宫同日 也作出回应，称“这不是由拜登决定的”。报道称，在华沙皇家城堡发表的讲话中，拜登在\
    谴责普京后表示“看在上帝的份上，这个人不能继续掌权了”。对此，路透社表示，该言论引发了华盛顿方面对局势升级的担忧，美国\
    一直避免直接对乌克兰进行军事干预，并明确表示不支持政权更迭。随后，一名白宫官员对拜登这番话进行解释，称拜登的言论并不\
    代表华盛顿政策的转变，其目的是让世界为乌克兰问题上的长期冲突做好准备。该官员称，拜登意为“不能 允许普京对其邻国或该地区\
    行使权力”，而不是在讨论普京在俄罗斯的权力或俄 罗斯的政权 更迭问题。同日，路透社称，被问及拜登这一言论时，俄罗斯总统新闻\
    秘书佩斯科夫作出回应，称“这不是由拜登决定的。俄罗斯总统是由俄罗斯人选举产生的”。此前 ，俄罗斯安全会议副主席梅德韦杰夫接受\
    俄媒采访时曾表示，俄特别军事行动既定目标包括实现乌克兰去军事化、去纳粹化、成为中立国家、不奉行反俄政策。俄开展特别军事行动\
    首先是因为其设定的目标未能通过外交方式实现。他表示，社会调查显示，四分之三的俄罗斯民众支持在乌开展特别军事行动。西方国家企\
    图通过制裁影响俄民众，煽动他们反对政府，结果适得其反。"
jieba.analyse.textrank(text, topK=20, withWeight=True, allowPOS=('ns', 'n', 'vn', 'v','nr', 'nt'))

## 结合主题的LDA算法
- `sklearn`库和`genism`库

In [ ]:
## 英文语料库
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
# Define the word list
word_list = [["running", "cars"],["better", "driving"]]

# Lemmatize list of words and join
def lemmatize_sentence(list):
    lemmatizer = WordNetLemmatizer()
    tagged_sentence = pos_tag(list)
    lemmatized_sentence = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in tagged_sentence]
    return lemmatized_sentence

# Lemmatize the word list
lemmatized_output = [lemmatize_sentence(word) for word in word_list]

In [ ]:
from gensim import corpora, models
from gensim.models import LdaModel

# Create the term dictionary of our corpus, where every unique term is assigned an index
dictionary = corpora.Dictionary(lemmatized_output)

# Convert list of documents (corpus) into Document Term Matrix using dictionary prepared above.
corpus = [dictionary.doc2bow(doc) for doc in lemmatized_output]

# Create a TF-IDF model from the corpus
tfidf = models.TfidfModel(corpus)

# Transform the corpus to TF-IDF
corpus_tfidf = tfidf[corpus]

In [ ]:
import matplotlib.pyplot as plt
from gensim.models import CoherenceModel
from tqdm import tqdm
# Create empty lists to store coherence and perplexity values
coherence_values = []
perplexity_values = []
range_topics = range(1, 6)

# Loop through different numbers of topics
for num_topics in tqdm(range_topics):
    # Train LDA model with the given number of topics
    ldamodel = LdaModel(corpus_tfidf, num_topics=num_topics, id2word=dictionary, passes=50)
    # Calculate coherence score
    coherence_model = CoherenceModel(model=ldamodel, texts=lemmatized_output, dictionary=dictionary, coherence='c_v')
    coherence = coherence_model.get_coherence()
    # Calculate perplexity
    perplexity = ldamodel.log_perplexity(corpus_tfidf)
    # Append coherence and perplexity values to the lists
    coherence_values.append(coherence)
    perplexity_values.append(perplexity)

# Plot the relationship between number of topics and coherence values
fig, ax1 = plt.subplots()
line1, = ax1.plot(range_topics, coherence_values, label='Coherence', color='blue')
ax1.set_xlabel('Number of Topics')
ax1.set_ylabel('Coherence', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Create a second y-axis for perplexity values
ax2 = ax1.twinx()
line2, = ax2.plot(range_topics, perplexity_values, label='Perplexity', color='red')
ax2.set_ylabel('Perplexity', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Set title and legend
plt.title('Coherence and Perplexity vs. Number of Topics')

# Combine legends
lines = [line1, line2]
labels = [l.get_label() for l in lines]
ax1.legend(lines, labels, loc='upper right')

# Show the plot
plt.show()

## 结合语义编码的KeyBert算法

In [ ]:
from keybert import KeyBERT
import jieba 

# model = KeyBERT('bert-base-chinese')
# text = "美国总统拜登当地时间26日又语出惊人。据路透社报道，拜登当天在波兰的演 讲中，称俄罗斯总统普京“不能继续掌权了”。\
# 白宫官员随后出来解释，克里姆林宫同日也作出回应，称“这不是由拜登决定的”。报道称，在华沙皇家城堡发表的讲话中，\
# 拜登在谴责普京后表示“看在上帝的份上，这个人不能继续掌权了”。对此，路透社表示，该言论引发了华盛顿方面对局势升级的担忧，\
# 美国一直避免直接对乌克兰进行军事干预，并明确表示不支持政权更迭。随后，一名白宫官员对拜登这番话进行解释，称拜登的言论并不代表华盛顿政策的转变，\
# 其目的是让世界为乌克兰问题上的长期冲突做好准备。该官员称，拜登意为“不能 允许普京对其邻国或该地区行使权力”，\
# 而不是在讨论普京在俄罗斯的权力或俄罗斯的政权 更迭问题。同日，路透社称，被问及拜登这一言论时，俄罗斯总统新闻秘书佩斯科夫作出回应，\
# 称“这不是由拜登决定的。俄罗斯总统是由俄罗斯人选举产生的”。此前，俄罗斯安全会议副主席梅德韦杰夫接受俄媒采访时曾表示，\
# 俄特别军事行动既定目标包括实现乌克兰去军事化、去纳粹化、成为中立国家、不奉行反俄政策。俄开展特别军事行动首先是因为其设定的目标未能通过外交方式实现。\
# 他表示，社会调查显示，四分之三的俄罗斯民众支持在乌开展特别军事行动。西方国家企图通过制裁影响俄民众，煽动他们反对政府，结果适得其反。"
# doc = " ".join(jieba.cut(text))

model = KeyBERT('bert-base-uncased')
doc = "I like it very much"
keywords = model.extract_keywords(doc, keyphrase_ngram_range=(1,2),  top_n=20)
keywords